# Solución de ecuaciones
<p><code>Python en Jupyter Notebook</code></p>
<p>Creado por <code>Giancarlo Ortiz</code> para el curso de <code>Métodos Numéricos</code></p>
<style type="text/css">
    .border {
        display: inline-block;
        border: solid 1px rgba(204, 204, 204, 0.4);
        border-bottom-color: rgba(187, 187, 187, 0.4);
        border-radius: 3px;
        box-shadow: inset 0 -1px 0 rgba(187, 187, 187, 0.4);
        background-color: inherit !important;
        vertical-align: middle;
        color: inherit !important;
        font-size: 11px;
        padding: 3px 5px;
        margin: 0 2px;
    }
</style>

## Busqueda de raíces por métodos abiertos
Los metodos abiertos, A diferencia de los métodos cerrados, calculan en cada iteración una aproxmación a la raíz, sin necesidad de verificar lo que sucede dentro de un intervalo.

## Agenda
1. Generalidades
1. Método de punto fijo
1. Método de Newton Raphson
1. Metodo de la secante

In [ ]:
# Importar módulos al cuaderno de jupyter
import math as m 
import numpy as np
import pylab as plt

# Definir e incluir nuevas funciones al cuaderno
def _buscar_intervalos(fun, ini, fin):
    """ Método para buscar intervalos en los que ocurra cambio de signo.

        ## Parámetros:
            fun (function): función para analizar.
            ini (int): inicio del análisis.
            fin (int): limite final del análisis.
        
        ## Devoluciones:
            I (list): Lista de tuplas con los intervalos donde hay un cero.
    """
    i = ini - 1 # Variable para contar iteraciones
    I = []      # Variable para almacenar los intervalos
    
    while i < fin:
        i += 1
        if fun(i) == 0:
            I.append((i, i))
        elif fun(i) * fun(i+1) < 0:
            I.append((i, i+1))
        else:
            pass
    return I

## 1. Metodos Abiertos
---
Los métodos abiertos calculan en cada iteración una aproximación a la raíz, pero no se ocupan de verificar si esta aproximación genera o no un intervalo que contenga una raíz; estos métodos utilizan una función auxiliar para predecir la raíz. Esta fórmula puede desarollarse como la sustitucion de un punto fijo en cada iteración.

## 2. Método de punto fijo
---
Para buscar una raíz de una función $f(x)=0$ a partir de un valor inicial, una tolerancia y un número maximo de iteraciones; se genera una funcion auxiliar $x=g(x)$; notese que para este caso no es necesario tener un intervalo.

\begin{align}
\tag{1} f(x) = 0 \\
\tag{2} f(x) + x = x \\
\tag{3} g(x) = x \\
\end{align}

* **NOTA:** El valor inicial se conoce como semilla

In [ ]:
# Defino el método del punto fijo
def _fijo(Func, Xmin, Xmax, Imax, Tmax):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (int): función que depende de una variable.
            Xmin (int): limite inferior de intervalo.
            Xmax (int): limite superior de intervalo.
            Imax (int): número máximo de iteraciones.
            Tmax (int): exponente de tolerancia máxima, T = 1e^Tmax.
        
        ## Devoluciones:
            Km (float): valor de x encontrado.
            No (int)  : iteraciones.
            ea (float): error absoluto final
            Li (list): lista de las soluciones, los errores absolutos y la tabla con la evoluvión de iteraciones.
    """
    Func + 

## 3. Método de Newton-Raphson
---
En matemáticas, el método de bisección es un algoritmo de búsqueda de raíces que trabaja dividiendo el intervalo a la mitad y seleccionando el subintervalo que tiene la raíz.

\begin{equation*}
f(x_1, x_2, ... , x_n) = g(x_1, x_2, ... , x_n) \\
\end{equation*}

### Ventajas:
* Metodo estable, siempre converge.
* Útil como aproximación inicial de otros métodos.

### Desventajas:
* No tiene en cuenta la magnitud de los valores de la función en las aproximaciones calculadas $x_n$, solo tiene en cuenta el signo de $f(x)$.
* Método linealmente convergente
* Orden de convergencia, $r = 1$.
* Tasa de convergencia $\mu = 0.5$

In [ ]:
# Defino el método Newton-Raphson
def _newton(Func, Xmin, Xmax, Imax, Tmax):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (int): función que depende de una variable.
            Xmin (int): limite inferior de intervalo.
            Xmax (int): limite superior de intervalo.
            Imax (int): número máximo de iteraciones.
            Tmax (int): exponente de tolerancia máxima, T = 1e^Tmax.
        
        ## Devoluciones:
            Km (float): valor de x encontrado.
            No (int)  : iteraciones.
            ea (float): error absoluto final
            Li (list): lista de las soluciones, los errores absolutos y la tabla con la evoluvión de iteraciones.
    """
    

## 4. Metodo de la Secante
---
### a. Ecuaciones lineales
Un caso particular de las ecuaciones algebraicas sucede cuando solo los dos primeros coeficientes son distintos de cero y la solucion para x es unica y trivial.

\begin{align}
a_0 + a_1 x & = 0, \quad a_1 \neq 0 \\
x & = \frac{-a_0}{a_1} \\
\end{align}

### b. Ecuaciones cuadraticas
Un caso particular de las ecuaciones algebraicas sucede cuando solo los tres primeros coeficientes son distintos de cero y las 2 soluciones para x no son compleja.

\begin{align}
a_0 + a_1 x + a_2 x^2 & = 0, \quad a_1 \neq 0 \\
x_1, x_2 & = \frac{-a_1 \pm \sqrt{a_1^2 - 4 a_2 a_0 }}{2 a_2} \\
\end{align}

In [ ]:
# Defino el método iterativo de la Secante
def _secante(Func, Xmin, Xmax, Imax, Tmax):
    """ Método de la Bisección para encontrar raíces en un intervalo.

        ## Parámetros:
            Func (int): función que depende de una variable.
            Xmin (int): limite inferior de intervalo.
            Xmax (int): limite superior de intervalo.
            Imax (int): número máximo de iteraciones.
            Tmax (int): exponente de tolerancia máxima, T = 1e^Tmax.
        
        ## Devoluciones:
            Km (float): valor de x encontrado.
            No (int)  : iteraciones.
            ea (float): error absoluto final
            Li (list): lista de las soluciones, los errores absolutos y la tabla con la evoluvión de iteraciones.
    """

---
## Mas Recursos

- [Propagación de errores](https://en.wikipedia.org/wiki/Rounding) (Wikipedia)
- [Método iterativo](https://es.wikipedia.org/wiki/M%C3%A9todo_iterativo) (Wikipedia)
- [Estabilidad numérica](https://es.wikipedia.org/wiki/Estabilidad_num%C3%A9rica) (Wikipedia)
- [Orden de convergencia](https://es.wikipedia.org/wiki/Orden_de_convergencia) (Wikipedia)
